In [1]:
import numpy as np

In [2]:
class Const:
    def __init__(self, value):
        self.value = value
        
    def evaluate(self):
        return self.value

    def backpropagate(self, gradient):
        pass
    
    def __str__(self):
        return str(self.value)

class Var:
    def __init__(self, name, init_value=0):
        self.value = init_value
        self.name = name
    
    def evaluate(self):
        self.gradient = 0.0
        return self.value
    
    def backpropagate(self, gradient):
        self.gradient += gradient
    
    def __str__(self):
        return self.name

class BinaryOperator:
    def __init__(self, a, b):
        self.a = a
        self.b = b

class Add(BinaryOperator):
    def evaluate(self):
        self.value = self.a.evaluate() + self.b.evaluate()
        return self.value
    
    def backpropagate(self, gradient):
        self.a.backpropagate(gradient)
        self.b.backpropagate(gradient)
    
    def __str__(self):
        return f"{self.a} + {self.b}"

class Mul(BinaryOperator):
    def evaluate(self):
        self.value = self.a.evaluate() * self.b.evaluate()
        return self.value

    def backpropagate(self, gradient):
        self.a.backpropagate(gradient * self.b.value)
        self.b.backpropagate(gradient * self.a.value)
    
    def __str__(self):
        return f"({self.a}) * ({self.b})"

class Function:
    def __init__(self, a):
        self.a = a
    
class Log(Function):
    def evaluate(self):
        self.value = np.log(self.a.evaluate())
        return self.value
    
    def backpropagate(self, gradient):
        self.a.backpropagate(gradient * 1.0 / self.a.value)
        
    def __str__(self):
        return f"log({self.a})"
    
class Logistic(Function):
    def evaluate(self):
        self.value = 1.0 / (1.0 + np.exp(-self.a.evaluate()))
        return self.value
    
    def backpropagate(self, gradient):
        self.a.backpropagate(gradient * self.value * (1.0 - self.value))
        
    def __str__(self):
        return f"s({self.a})"

In [3]:
alpha = 1

w0 = Var('w0', init_value=0.0)
w1 = Var('w1', init_value=0.0)
b = Var('b', init_value=0.0)

Xs = [
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
]
ys = [0, 0, 0, 1]

minus = Const(-1)
one = Const(1)
for epoch in range(10000):
    loss = Const(0)
    for xval, yval in zip(Xs,ys):
        x0 = Const(xval[0])
        x1 = Const(xval[1])
        y = Const(yval)
        y_pred = Logistic(Add(b, Add(Mul(x0, w0), Mul(x1, w1))))
        loss = Add(loss, 
                   Mul(minus, 
                       Add(Mul(y, Log(y_pred)), 
                           Mul(Add(one, Mul(minus, y)), 
                               Log(Add(one, Mul(minus, y_pred))))
                          )
                      )
                  )
    loss = Mul(Const(0.25), loss)
    loss.evaluate()
    loss.backpropagate(1.0)
    w0.value -= alpha * w0.gradient
    w1.value -= alpha * w1.gradient
    b.value -= alpha * b.gradient

In [4]:
w0.value

12.074945269923951

In [5]:
w1.value

12.074945269923951

In [6]:
b.value

-18.281070367299836